In [20]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from scipy.special import kl_div
from keras.models import load_model
from pathlib import Path
params = {'results_path': 'results',
          'data_path': 'data',
          'nfp': 2,
          'test_size': 0.2,
          'random_state': 42,
          'n_components': 5,
          'n_samples': 1000,
          'model': 'nn'
          }
this_path = str(os.path.abspath(''))
general_results_path = os.path.join(this_path, params['results_path'])
results_path = os.path.join(general_results_path, f'nfp{params["nfp"]}')
os.makedirs(results_path, exist_ok=True)
model_path = os.path.join(params['results_path'], f"nfp{params['nfp']}", f"nn_qsc_nfp{params['nfp']}_model{params['model']}.h5")
scaler_x_path = os.path.join(params['results_path'], f"nfp{params['nfp']}", f"nn_qsc_nfp{params['nfp']}_scaler_x.pkl")
scaler_y_path = os.path.join(params['results_path'], f"nfp{params['nfp']}", f"nn_qsc_nfp{params['nfp']}_scaler_y.pkl")
model = load_model(model_path)
scaler_x = joblib.load(scaler_x_path)
scaler_y = joblib.load(scaler_y_path)

In [17]:
# Load data
filename = os.path.join(this_path, params['data_path'], f'qsc_out.random_scan_nfp{params["nfp"]}.parquet')
df = pd.read_parquet(filename)

# Byte order fix
for column in df.columns:
    if df[column].dtype.byteorder == '>':
        df[column] = df[column].values.byteswap().newbyteorder()

# Drop ysum column if exists
if 'ysum' in df.columns:
    df = df.drop(columns='ysum')

# Define columns
x_columns = [col for col in df.columns if col.startswith('x')]
y_columns = [col for col in df.columns if col.startswith('y')]

# Split data
Y = df[x_columns].values
X = df[y_columns].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=params['test_size'], random_state=params['random_state'])

In [3]:
# GMM for all data
gmm_all = GaussianMixture(n_components=params['n_components']).fit(df.values)

# GMM for input data
gmm_input = GaussianMixture(n_components=params['n_components']).fit(X)

In [18]:
from scipy.stats import entropy

# Generate new samples
samples_all = gmm_all.sample(params['n_samples'])[0]
samples_input = gmm_all.sample(len(df.values))[0]

def kl_div(p, q):
    """Compute the Kullback-Leibler divergence between two distributions."""
    return entropy(p, q)

# Compute histograms
hist_df_all, _ = np.histogram(df.values, bins=100, density=True)
hist_samples_all, _ = np.histogram(samples_all, bins=100, density=True)

# Compute KL divergence
kl_div_all = kl_div(hist_df_all+1e-10, hist_samples_all+1e-10)

print(f"KL divergence (all data): {kl_div_all}")

KL divergence (all data): 1.6817099982342174


In [25]:
# Predict using generated samples
predictions_all = model.predict(samples_all[:, len(x_columns):])
predictions_input = model.predict(samples_input[:, len(x_columns):])

21875/21875 [==============================] - 18s 820us/step


In [27]:
print(Y_test.shape)
print(predictions_input.shape)
print(predictions_all.shape)

(140000, 8)
(700000, 8)
(1000, 8)


In [26]:
from sklearn.metrics import mean_absolute_error

# Process and output best predictions
# Note: you need to define how to determine the "best" output
# Assuming Y_test is available and corresponding to samples_all and samples_input

# Calculate MAE for all data samples
mae_all = mean_absolute_error(Y_test, predictions_all)

# Calculate MAE for input data samples
mae_input = mean_absolute_error(Y_test, predictions_input)

print(f"MAE for all data samples: {mae_all}")
print(f"MAE for input data samples: {mae_input}")

# Determine which set of predictions has the lowest MAE
best_predictions = predictions_all if mae_all < mae_input else predictions_input
print("Best predictions:", best_predictions)


ValueError: Found input variables with inconsistent numbers of samples: [140000, 1000]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture
from scipy.stats import entropy

def kl_div(p, q):
    """Compute the Kullback-Leibler divergence between two distributions."""
    return entropy(p, q)

# Take a smaller subset of data for simplicity
data_subset = np.random.choice(df.values.flatten(), 5000)
samples_subset = np.random.choice(samples_all.flatten(), 5000)

# Define bins
bins = np.linspace(min(data_subset.min(), samples_subset.min()), 
                   max(data_subset.max(), samples_subset.max()), 100)

# Method 1: Histogram
hist_data, _ = np.histogram(data_subset, bins=bins, density=True)
hist_samples, _ = np.histogram(samples_subset, bins=bins, density=True)

# Method 2: Kernel Density Estimation (KDE)
kde_data = KernelDensity(bandwidth=1.0, kernel='gaussian')
kde_data.fit(data_subset[:, None])
logprob_data = kde_data.score_samples(bins[:, None])

kde_samples = KernelDensity(bandwidth=1.0, kernel='gaussian')
kde_samples.fit(samples_subset[:, None])
logprob_samples = kde_samples.score_samples(bins[:, None])

# Method 3: Gaussian Mixture Models (GMM)
gmm_data = GaussianMixture(n_components=2)
gmm_data.fit(data_subset[:, None])
logprob_gmm_data = gmm_data.score_samples(bins[:, None])

gmm_samples = GaussianMixture(n_components=2)
gmm_samples.fit(samples_subset[:, None])
logprob_gmm_samples = gmm_samples.score_samples(bins[:, None])

# KL divergence
kl_div_hist = kl_div(hist_data+1e-10, hist_samples+1e-10)
kl_div_kde = kl_div(np.exp(logprob_data), np.exp(logprob_samples))
kl_div_gmm = kl_div(np.exp(logprob_gmm_data), np.exp(logprob_gmm_samples))

# Print KL divergences
print(f"KL divergence (Histogram): {kl_div_hist}")
print(f"KL divergence (KDE): {kl_div_kde}")
print(f"KL divergence (GMM): {kl_div_gmm}")

# Plot the results
plt.figure(figsize=(12, 8))

plt.subplot(311)
plt.plot(bins, hist_data, label='Data')
plt.plot(bins, hist_samples, label='Samples')
plt.title(f'Histogram (KL divergence = {kl_div_hist:.2f})')
plt.legend()

plt.subplot(312)
plt.plot(bins, np.exp(logprob_data), label='Data')
plt.plot(bins, np.exp(logprob_samples), label='Samples')
plt.title(f'KDE (KL divergence = {kl_div_kde:.2f})')
plt.legend()

plt.subplot(313)
plt.plot(bins, np.exp(logprob_gmm_data), label='Data')
plt.plot(bins, np.exp(logprob_gmm_samples), label='Samples')
plt.title(f'GMM (KL divergence = {kl_div_gmm:.2f})')
plt.legend()

plt.tight_layout()
plt.show()